In [1]:
import pandas as pd

## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb). 
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке 

In [29]:
# загрузка данных из файлов
movies_dataset = pd.read_csv('../../datasets/tmdb_5000_movies.csv')
credits_dataset = pd.read_csv('../../datasets/tmdb_5000_credits.csv')

# объединение данных с указанием суффиксов для дублирующихся столбцов
dataset = movies_dataset.merge(credits_dataset, left_on='id', right_on='movie_id', suffixes=('_1', '_2'))

# фильтрация по релизу
dataset = dataset[dataset['status'] == 'Released']

# удаление фильмов с пропусками в столбцах 'overview', 'genres', 'keywords'
dataset = dataset.dropna(subset=['overview', 'genres', 'keywords'])

# удаление дублирующихся столбцов 'movie_id' и 'title_y'
dataset.drop(columns=['movie_id', 'title_2'], inplace=True)

# переименование столбца 'title_1' в 'title'
dataset.rename(columns={'title_1': 'title'}, inplace=True)

print("Количество фильмов оставшихся в выборке:", len(dataset))

Количество фильмов оставшихся в выборке: 4792


## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

# замена NaN в описании фильма на пустой символ
dataset['overview'].fillna('')

# создание экземпляра TfidfVectorizer с удалением английских стоп-слов
vectorizer = TfidfVectorizer(stop_words='english',  max_features=10000)

# расчёт Tf-Idf для описания фильма
tfidf_matrix = vectorizer.fit_transform(dataset['overview'] + ' ' + dataset['keywords'])

print("размер матрицы Tf-Idf:", tfidf_matrix.shape)

размер матрицы Tf-Idf: (4792, 10000)


## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

> Получившиеся файлы `distance.csv` и `movies.csv` пушить в GitLab не нужно!

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

# рассчет cosine similarity между фильмами
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# создание DataFrame с расстояниями
movies_ids = dataset['id']
cosine_sim_df = pd.DataFrame(cosine_sim, index=movies_ids, columns=movies_ids)

# сохранение DataFrame с расстояниями в файл
cosine_sim_df.to_csv('../assets/distance.csv')

# сохранение объединенного датасета с фильмами в файл
dataset.to_csv('../assets/movies.csv', index=False)